# Imports

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import random
import time
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_curve

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

print("All necessary libraries imported.")

In [ ]:
SEED = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

WINDOW_SIZE_FRAMES = 150
STEP_SIZE_FRAMES = 30
BATCH_SIZE = 128

INPUT_FEATURES = 90 # 36 (normalized poses) + 36 (velocities) + 18 (orientation angles)
HIDDEN_SIZE = 128
NUM_LAYERS = 2
NUM_CLASSES = 2
NUM_EPOCHS = 10

def set_seed(seed_value: int):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print(f"Global seed set to {seed_value}")

def load_and_label_files(data_root: str) -> pd.DataFrame:

    all_csv_files = glob(os.path.join(data_root, '**', '*.csv'), recursive=True)
    labeled_data = [{'path': file_path, 'label': 1 if 'Potential_shoplifter' in file_path else 0} for file_path in all_csv_files]
    df_files = pd.DataFrame(labeled_data)
    print(f"Found {len(df_files)} total CSV files.")
    if not df_files.empty:
        print("\nValue counts for labels:")
        print(df_files['label'].value_counts())
    return df_files

def parse_poses_from_string(poses_str: str) -> np.ndarray:
    
    try:
        return np.array(json.loads(poses_str)).reshape(18, 2)
    except (json.JSONDecodeError, ValueError):
        return np.zeros((18, 2))

# def extract_features(csv_path: str) -> np.ndarray | None:
  
#     df = pd.read_csv(csv_path)
#     if df.empty or 'POSES' not in df.columns:
#         return None

#     raw_poses = np.array(df['POSES'].apply(parse_poses_from_string).tolist()) 
   
#     #raw_poses shape: (T, 18, 2) => (frames, keypoints, (x,y))
#     neck_positions = raw_poses[:, 1:2, :] 

#     normalized_poses = raw_poses - neck_positions # We normalize the poses with respect to the neck position

#     velocities = np.diff(raw_poses, axis=0, prepend=raw_poses[0:1]) # Compute velocities

#     neck_trajectory = raw_poses[:, 1, :] # Extract neck trajectory why? Because the neck is a key point for understanding the body's movement.

#     deltas = np.diff(neck_trajectory, axis=0, prepend=[neck_trajectory[0]])  # Compute deltas why? Because the deltas represent the change in position of the neck over time.

#     orientation_angles_deg = np.degrees(np.arctan2(deltas[:, 1], deltas[:, 0])) # Compute orientation angles why? Because the orientation angles help in understanding the direction of movement.
    
#     # we only use neck orientation as the key point for understanding the body's movement why? Because the neck is a key point for understanding the body's movement
#     # but it shows limitation so maybe adding other keypoints like shoulders or hips could provide more context.

#     return np.concatenate([
#         normalized_poses.reshape(raw_poses.shape[0], -1),
#         velocities.reshape(raw_poses.shape[0], -1),
#         orientation_angles_deg.reshape(-1, 1)
#     ], axis=1)

def extract_features(csv_path: str) -> np.ndarray | None:

    df = pd.read_csv(csv_path)

    raw_poses = np.array(df['POSES'].apply(parse_poses_from_string).tolist())
    
    neck_positions = raw_poses[:, 1:2, :]  
    normalized_poses = raw_poses - neck_positions
    velocities = np.diff(raw_poses, axis=0, prepend=raw_poses[0:1])

    all_orientation_angles = []
    for i in range(raw_poses.shape[1]):
        joint_trajectory = raw_poses[:, i, :]
        
        deltas = np.diff(joint_trajectory, axis=0, prepend=[joint_trajectory[0]])
        
        orientation_deg = np.degrees(np.arctan2(deltas[:, 1], deltas[:, 0]))
        all_orientation_angles.append(orientation_deg)
        
    all_orientations_feature = np.stack(all_orientation_angles, axis=1)

    return np.concatenate([
        normalized_poses.reshape(raw_poses.shape[0], -1),  # Shape: (T, 36)
        velocities.reshape(raw_poses.shape[0], -1),      # Shape: (T, 36)
        all_orientations_feature                           # Shape: (T, 18)
    ], axis=1)


set_seed(SEED)
print(f"Using device: {device}")
data_root = r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter'
#data_root=r'/kaggle/input/skeleton-poses-normalvsshoplifter'
df_files = load_and_label_files(data_root)

df_files.head()

In [ ]:
def get_group_id(file_path: str) -> str:

    return os.path.basename(file_path).replace('.csv', '')[-23:]

def create_sliding_windows(sequences: list, labels: list, window_size: int, step_size: int):

    windowed_sequences, windowed_labels = [], []

    for i, seq in enumerate(sequences):
        # Sequences shorter than window_size are dropped (no padding)
        # Overlap amount = window_size - step_size (here 150 - 30 = 120 frames overlap → heavy redundancy).

        if seq.shape[0] >= window_size:
            for start in range(0, seq.shape[0] - window_size + 1, step_size):

                windowed_sequences.append(seq[start:start + window_size])

                windowed_labels.append(labels[i])

    return np.array(windowed_sequences), np.array(windowed_labels)

print(" Processing all CSV files into feature sequences ")

all_sequences, all_labels, all_groups, all_paths = [], [], [], []

for _, row in tqdm(df_files.iterrows(), total=len(df_files), desc="Extracting Features"):

    features = extract_features(row['path'])

    if features is not None and len(features) > 0:
        
        all_sequences.append(features)
        all_labels.append(row['label'])
        all_groups.append(get_group_id(row['path']))
        all_paths.append(row['path'])

print("\n Splitting data by group to prevent leakage ")

gss_train_temp = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_group_indices, temp_group_indices = next(gss_train_temp.split(all_sequences, all_labels, all_groups))

train_sequences = [all_sequences[i] for i in train_group_indices]
train_labels = [all_labels[i] for i in train_group_indices]

temp_sequences = [all_sequences[i] for i in temp_group_indices]
temp_labels = [all_labels[i] for i in temp_group_indices]
temp_groups = np.array(all_groups)[temp_group_indices]
temp_paths = [all_paths[i] for i in temp_group_indices]

print("\n Creating datasets: augmented for training, and both windowed & full-length for val/test ")

X_train_windowed, y_train_windowed = create_sliding_windows(
    train_sequences, train_labels, WINDOW_SIZE_FRAMES, STEP_SIZE_FRAMES
)

gss_val_test = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
val_indices, test_indices = next(gss_val_test.split(temp_sequences, temp_labels, temp_groups))

X_val_long = [temp_sequences[i] for i in val_indices]
y_val_long = [temp_labels[i] for i in val_indices]
X_test_long = [temp_sequences[i] for i in test_indices]
y_test_long = [temp_labels[i] for i in test_indices]
test_paths_long = [temp_paths[i] for i in test_indices]

X_val_windowed, y_val_windowed = create_sliding_windows(X_val_long, y_val_long, WINDOW_SIZE_FRAMES, STEP_SIZE_FRAMES)
X_test_windowed, y_test_windowed = create_sliding_windows(X_test_long, y_test_long, WINDOW_SIZE_FRAMES, STEP_SIZE_FRAMES)

X_train_tensor = torch.from_numpy(X_train_windowed).float()
y_train_tensor = torch.from_numpy(y_train_windowed).long()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

g = torch.Generator()
g.manual_seed(SEED)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=g)

print("\n Data Preparation Complete ")
print(f"Total original videos processed: {len(all_sequences)}")
print(f"Training on {len(X_train_tensor)} augmented windows (from {len(train_sequences)} videos).")
print(f"Validating on {len(X_val_long)} full-length videos.")
print(f"Testing on {len(X_test_long)} full-length videos.")

In [ ]:
class RealTimeClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.5):
        super(RealTimeClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.dropout = nn.Dropout(dropout_rate)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)
        attention_scores = self.attention(out)
        attention_weights = torch.softmax(attention_scores, dim=1)
        context_vector = torch.sum(attention_weights * out, dim=1)
        context_vector_dropped = self.dropout(context_vector)
        return self.fc(context_vector_dropped)

class FocalLoss(nn.Module):
    def __init__(self, alpha: torch.Tensor, gamma: float = 2.0, reduction: str = 'mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha[targets] * (1 - pt)**self.gamma * ce_loss)
        return focal_loss.mean() if self.reduction == 'mean' else focal_loss.sum()

def evaluate_and_report_on_test_set(
    model: nn.Module,
    test_sequences: list[np.ndarray],
    test_labels: list[int],
    device: torch.device
):
    model.eval()
    all_window_preds, all_window_labels = [], []
    video_suspicion_scores = []

    with torch.no_grad():
        for i, long_seq_np in enumerate(tqdm(test_sequences, desc="Evaluating Test Set")):
            y_true_video = test_labels[i]
            seq_windows_tensors = []
            if long_seq_np.shape[0] >= WINDOW_SIZE_FRAMES:
                for start in range(0, long_seq_np.shape[0] - WINDOW_SIZE_FRAMES + 1, STEP_SIZE_FRAMES):
                    window = long_seq_np[start:start + WINDOW_SIZE_FRAMES]
                    seq_windows_tensors.append(torch.from_numpy(window))
            if not seq_windows_tensors:
                video_suspicion_scores.append(0.0)
                continue
            windows_batch = torch.stack(seq_windows_tensors).float().to(device)
            outputs = model(windows_batch)
            _, predicted_windows = torch.max(outputs, 1)
            all_window_preds.extend(predicted_windows.cpu().numpy())
            all_window_labels.extend([y_true_video] * len(predicted_windows))
            probabilities = torch.softmax(outputs, dim=1)[:, 1]
            def aggregate_window_probs(probabilities, mode="mean", k=3, p0=0.6):
                probs = probabilities.cpu().numpy()
                if mode == "mean":
                    return probs.mean()
                if mode == "max":
                    return probs.max()
                if mode == "median":
                    return np.median(probs)
                if mode == "topk_mean":
                    k = min(k, len(probs))
                    return np.sort(probs)[-k:].mean()
                if mode == "prop_above":
                    return (probs >= p0).mean()
                raise ValueError(f"Unknown mode {mode}")
            avg_suspicion_score = aggregate_window_probs(probabilities, mode="mean")
            video_suspicion_scores.append(avg_suspicion_score)

    plot_report_and_matrix(all_window_labels, all_window_preds, "Test Set Evaluation (Per-Window)")

    simple_accuracy = evaluate_realistically(model, test_sequences, test_labels, device, WINDOW_SIZE_FRAMES, STEP_SIZE_FRAMES)
    print(f"\n Test Set Evaluation (Simple Per-Video) ")
    print(f"Accuracy (if any window is 1, predict 1): {simple_accuracy:.2f}%")

    y_true_video_labels = test_labels
    precisions, recalls, thresholds = precision_recall_curve(y_true_video_labels, video_suspicion_scores)
    f1_scores = np.nan_to_num(2 * (precisions * recalls) / (precisions + recalls))
    best_threshold = thresholds[np.argmax(f1_scores)]
    y_pred_thresholded = [1 if score >= best_threshold else 0 for score in video_suspicion_scores]
    plot_report_and_matrix(y_true_video_labels, y_pred_thresholded, f"Test Set Evaluation (Optimal Threshold: {best_threshold:.2f})")

def plot_report_and_matrix(y_true, y_pred, title):
    print(f"\n {title} ")
    print(classification_report(y_true, y_pred, target_names=['Normal', 'Potential Shoplifter'], zero_division=0))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=['Normal', 'Potential Shoplifter'], yticklabels=['Normal', 'Potential Shoplifter'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.show()

def evaluate_realistically(model, long_sequences, long_labels, device, window_size, step_size):
    model.eval()
    y_pred_final = []
    with torch.no_grad():
        for long_seq_np in long_sequences:
            seq_windows_tensors = []
            if long_seq_np.shape[0] >= window_size:
                for start in range(0, long_seq_np.shape[0] - window_size + 1, step_size):
                    seq_windows_tensors.append(torch.from_numpy(long_seq_np[start:start + window_size]))
            if not seq_windows_tensors:
                final_video_prediction = 0
            else:
                outputs = model(torch.stack(seq_windows_tensors).float().to(device))
                final_video_prediction = 1 if 1 in torch.max(outputs, 1)[1] else 0
            y_pred_final.append(final_video_prediction)
    return accuracy_score(long_labels, y_pred_final) * 100

print("Model architecture, loss functions, and evaluation helpers are defined.")

In [ ]:
def run_online_training_trial(
    model: nn.Module,
    train_loader: DataLoader,
    val_windowed_loader: DataLoader,
    val_long_sequences: list[np.ndarray],
    val_long_labels: list[int],
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    num_epochs: int,
    model_save_path_window: str,
    model_save_path_realistic: str,
    device: torch.device
):
    best_val_acc_window = 0.0
    best_val_acc_realistic = 0.0
    history = {'train_loss': [], 'val_acc_window': [], 'val_acc_realistic': []}

    for epoch in range(num_epochs):
        start_time = time.time()

        model.train()
        train_loss = 0.0
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} | Training")
        for sequences, labels in train_pbar:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_pbar.set_postfix(loss=f"{loss.item():.4f}")

        avg_train_loss = train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        model.eval()
        correct_window, total_window = 0, 0
        with torch.no_grad():
            for sequences, labels in val_windowed_loader:
                sequences, labels = sequences.to(device), labels.to(device)
                outputs = model(sequences)
                _, predicted = torch.max(outputs.data, 1)
                total_window += labels.size(0)
                correct_window += (predicted == labels).sum().item()

        epoch_acc_val_window = 100 * correct_window / total_window
        history['val_acc_window'].append(epoch_acc_val_window)

        epoch_acc_val_realistic = evaluate_realistically(
            model, val_long_sequences, val_long_labels, device, WINDOW_SIZE_FRAMES, STEP_SIZE_FRAMES
        )
        history['val_acc_realistic'].append(epoch_acc_val_realistic)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Window Val Acc: {epoch_acc_val_window:.2f}% | Realistic Val Acc: {epoch_acc_val_realistic:.2f}%")

        if epoch_acc_val_window > best_val_acc_window:
            best_val_acc_window = epoch_acc_val_window
            torch.save(model.state_dict(), model_save_path_window)
            print(f" New best WINDOW model saved (Val Acc: {best_val_acc_window:.2f}%)")

        if epoch_acc_val_realistic > best_val_acc_realistic:
            best_val_acc_realistic = epoch_acc_val_realistic
            torch.save(model.state_dict(), model_save_path_realistic)
            print(f" New best REALISTIC model saved (Val Acc: {best_val_acc_realistic:.2f}%)")

    print("\n Finished Training Trial ")
    return history

print(" Trial 1: Training with Standard Cross-Entropy Loss ")

set_seed(SEED)

X_val_tensor = torch.from_numpy(X_val_windowed).float()
y_val_tensor = torch.from_numpy(y_val_windowed).long()
val_windowed_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_windowed_loader = DataLoader(val_windowed_dataset, batch_size=BATCH_SIZE, shuffle=False)

ce_model = RealTimeClassifier(INPUT_FEATURES, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)

ce_criterion = nn.CrossEntropyLoss()
ce_optimizer = optim.Adam(ce_model.parameters(), lr=0.001, weight_decay=1e-4)

ce_model_path_window = 'best_ce_model_window.pth'
ce_model_path_realistic = 'best_ce_model_realistic.pth'

ce_history = run_online_training_trial(
    model=ce_model,
    train_loader=train_loader,
    val_windowed_loader=val_windowed_loader,
    val_long_sequences=X_val_long,
    val_long_labels=y_val_long,
    criterion=ce_criterion,
    optimizer=ce_optimizer,
    num_epochs=NUM_EPOCHS,
    model_save_path_window=ce_model_path_window,
    model_save_path_realistic=ce_model_path_realistic,
    device=device
)

In [ ]:
def plot_history(history: dict, trial_name: str):
    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Training Loss', color='tab:red')
    ax1.plot(history['train_loss'], 'r-', label='Training Loss')
    ax1.tick_params(axis='y', labelcolor='tab:red')
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.set_ylabel('Validation Accuracy (%)', color='tab:blue')
    ax2.plot(history['val_acc_window'], 'b--', label='Window Val Acc')
    ax2.plot(history['val_acc_realistic'], 'g-.', label='Realistic Val Acc')
    ax2.tick_params(axis='y', labelcolor='tab:blue')

    fig.tight_layout()
    fig.suptitle(f'{trial_name} - Training History', y=1.03)
    fig.legend(loc='upper right', bbox_to_anchor=(0.9, 0.9))
    plt.show()

print(" Plotting Training History for Trial 1 (Cross-Entropy) ")
plot_history(ce_history, "Cross-Entropy Loss")

print("\n Loading best model for final evaluation on the Test Set ")
set_seed(SEED)

final_ce_model = RealTimeClassifier(INPUT_FEATURES, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)
final_ce_model.load_state_dict(torch.load(ce_model_path_realistic))
print(f"Model loaded successfully from: {ce_model_path_realistic}")

evaluate_and_report_on_test_set(
    model=final_ce_model,
    test_sequences=X_test_long,
    test_labels=y_test_long,
    device=device
)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
import torch
import torch.nn as nn


# --- Plotting Function for Single Windows ---
def plot_trajectory_window_on_ax(ax: plt.Axes, csv_path: str, start_frame: int, window_size: int, title: str):
    ax.set_aspect('equal', adjustable='box')
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        ax.set_title("File Not Found")
        return
    
    end_frame = start_frame + window_size
    if df.empty or 'POSES' not in df.columns or len(df) < end_frame:
        ax.set_title(f"No data at frame {start_frame}")
        return

    raw_poses = np.array(df['POSES'].iloc[start_frame:end_frame].apply(parse_poses_from_string).tolist())
    neck_trajectory = raw_poses[:, 1, :]
    neck_trajectory = neck_trajectory[np.any(neck_trajectory != 0, axis=1)]

    if neck_trajectory.shape[0] > 1:
        ax.plot(neck_trajectory[:, 0], neck_trajectory[:, 1], color='red', linewidth=2.5)
        ax.plot(neck_trajectory[0, 0], neck_trajectory[0, 1], 'go', markersize=10, label='Start')
        ax.plot(neck_trajectory[-1, 0], neck_trajectory[-1, 1], 'ro', markersize=10, label='End')
        ax.set_title(title, fontsize=10)
        ax.set_xlabel('x (pixels)')
        ax.set_ylabel('y (pixels)')
        ax.legend(fontsize='small')
    else:
        ax.set_title(f"No valid data in window\nstarting at frame {start_frame}")


def find_all_failure_windows_in_test_set(model, test_videos, test_labels, test_paths):
    """
    Iterates through the entire test set, finds every misclassified window,
    and returns them categorized as False Positives or False Negatives.
    """
    model.eval()
    fp_windows = [] # List to store (csv_path, start_frame)
    fn_windows = [] # List to store (csv_path, start_frame)

    print(f"Analyzing {len(test_videos)} videos in the test set...")
    
    # Iterate through each video in the test set
    for video_idx, video_data in enumerate(test_videos):
        true_label = test_labels[video_idx]
        csv_path = test_paths[video_idx]
        
        # Iterate through the windows of the current video
        for start in range(0, video_data.shape[0] - WINDOW_SIZE_FRAMES + 1, STEP_SIZE_FRAMES):
            # The `video_data` is already the feature-extracted numpy array
            feature_window = video_data[start : start + WINDOW_SIZE_FRAMES]
            
            with torch.no_grad():
                input_tensor = torch.from_numpy(feature_window).unsqueeze(0).float().to(device)
                output = model(input_tensor)
                prediction = torch.max(output, 1)[1].item()
            
            if prediction != true_label:
                window_info = (csv_path, start)
                if true_label == 0: # False Positive
                    fp_windows.append(window_info)
                else: # False Negative
                    fn_windows.append(window_info)
                    
    return fp_windows, fn_windows

print("\n--- Finding All Misclassified Windows in the Test Set ---")
fp_windows, fn_windows = find_all_failure_windows_in_test_set(
    final_ce_model, X_test_long, y_test_long, test_paths_long
)
print(f"Found {len(fp_windows)} total False Positive windows across all 'Normal' videos.")
print(f"Found {len(fn_windows)} total False Negative windows across all 'Shoplifter' videos.")

if not fp_windows and not fn_windows:
    print("\nExcellent! No misclassified windows found anywhere in the test set.")
else:
    num_rows = max(len(fp_windows), len(fn_windows))
    fig, axes = plt.subplots(num_rows, 2, figsize=(16, 6 * num_rows), squeeze=False)
    fig.suptitle("Comprehensive Analysis of All Misclassified Windows in Test Set", fontsize=20, y=0.99)

    axes[0, 0].set_title("False Positives\n(Normal Windows -> Predicted Shoplifter)", fontsize=16, pad=25)
    axes[0, 1].set_title("False Negatives\n(Shoplifter Windows -> Predicted Normal)", fontsize=16, pad=25)

    # Plot all False Positive windows
    for i in range(num_rows):
        ax = axes[i, 0]
        if i < len(fp_windows):
            csv_path, start_frame = fp_windows[i]
            title = f"{os.path.basename(csv_path)}\n(Frames {start_frame}-{start_frame+WINDOW_SIZE_FRAMES})"
            plot_trajectory_window_on_ax(ax, csv_path, start_frame, WINDOW_SIZE_FRAMES, title)
        else:
            ax.set_visible(False)

    # Plot all False Negative windows
    for i in range(num_rows):
        ax = axes[i, 1]
        if i < len(fn_windows):
            csv_path, start_frame = fn_windows[i]
            title = f"{os.path.basename(csv_path)}\n(Frames {start_frame}-{start_frame+WINDOW_SIZE_FRAMES})"
            plot_trajectory_window_on_ax(ax, csv_path, start_frame, WINDOW_SIZE_FRAMES, title)
        else:
            ax.set_visible(False)
            
    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

In [ ]:
print(" Trial 2: Training with Focal Loss ")

set_seed(SEED)

focal_model = RealTimeClassifier(INPUT_FEATURES, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)

total_videos = 88 + 64
alpha_class_0 = 64 / total_videos
alpha_class_1 = 88 / total_videos
alpha_tensor = torch.tensor([alpha_class_0, alpha_class_1]).to(device)

print(f"Calculated alpha weights for Focal Loss: {alpha_tensor.cpu().numpy()}")

focal_criterion = FocalLoss(alpha=alpha_tensor, gamma=3.0)
focal_optimizer = optim.AdamW(focal_model.parameters(), lr=0.00001, weight_decay=1e-4)

focal_model_path_window = 'best_focal_model_window.pth'
focal_model_path_realistic = 'best_focal_model_realistic.pth'

focal_history = run_online_training_trial(
    model=focal_model,
    train_loader=train_loader,
    val_windowed_loader=val_windowed_loader,
    val_long_sequences=X_val_long,
    val_long_labels=y_val_long,
    criterion=focal_criterion,
    optimizer=focal_optimizer,
    num_epochs=50, #NUM_EPOCHS
    model_save_path_window=focal_model_path_window,
    model_save_path_realistic=focal_model_path_realistic,
    device=device
)

In [ ]:
def get_model_predictions_for_test_set(model: nn.Module, test_sequences: list, test_labels: list, device: torch.device) -> list:
    model.eval()
    video_suspicion_scores = []

    with torch.no_grad():
        for long_seq_np in test_sequences:
            seq_windows_tensors = []
            if long_seq_np.shape[0] >= WINDOW_SIZE_FRAMES:
                for start in range(0, long_seq_np.shape[0] - WINDOW_SIZE_FRAMES + 1, STEP_SIZE_FRAMES):
                    window = long_seq_np[start:start + WINDOW_SIZE_FRAMES]
                    seq_windows_tensors.append(torch.from_numpy(window))
            if not seq_windows_tensors:
                video_suspicion_scores.append(0.0)
                continue
            windows_batch = torch.stack(seq_windows_tensors).float().to(device)
            outputs = model(windows_batch)
            probabilities = torch.softmax(outputs, dim=1)[:, 1]
            avg_suspicion_score = probabilities.mean().item()
            video_suspicion_scores.append(avg_suspicion_score)

    precisions, recalls, thresholds = precision_recall_curve(test_labels, video_suspicion_scores)
    f1_scores = np.nan_to_num(2 * (precisions * recalls) / (precisions + recalls))
    best_threshold = thresholds[np.argmax(f1_scores)]
    y_pred_thresholded = [1 if score >= best_threshold else 0 for score in video_suspicion_scores]
    return y_pred_thresholded

print(" Plotting Training History for Trial 2 (Focal Loss) ")
plot_history(focal_history, "Focal Loss")

print("\n Loading best Focal Loss model for final evaluation on the Test Set ")
set_seed(SEED)

final_focal_model = RealTimeClassifier(INPUT_FEATURES, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)
final_focal_model.load_state_dict(torch.load(focal_model_path_realistic))
print(f"Model loaded successfully from: {focal_model_path_realistic}")

evaluate_and_report_on_test_set(
    model=final_focal_model,
    test_sequences=X_test_long,
    test_labels=y_test_long,
    device=device
)

print("\n Visualizing Failure Cases for Trial 2 (Focal Loss) ")

focal_test_predictions = get_model_predictions_for_test_set(final_focal_model, X_test_long, y_test_long, device)
focal_failure_indices = [i for i, (pred, true) in enumerate(zip(focal_test_predictions, y_test_long)) if pred != true]
fn_failures_focal = [i for i in focal_failure_indices if y_test_long[i] == 1]
fp_failures_focal = [i for i in focal_failure_indices if y_test_long[i] == 0]

print(f"Identified {len(focal_failure_indices)} total failure cases for the Focal Loss model.")
print(f" - {len(fn_failures_focal)} False Negatives (Missed Shoplifters)")
print(f" - {len(fp_failures_focal)} False Positives (Incorrectly Flagged Normals)")

plt.tight_layout(rect=[0, 0.03, 1, 0.93])
plt.show()